# 0. Prep the Environment

In [ ]:
import os, gzip, glob, shutil, pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, f_oneway
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import plotly.express as px
import seaborn as sns
from sklearn import decomposition

In [ ]:
for gz_fn in glob.glob('../data/*.gz'):
    if not os.path.exists(gz_fn[:-3]):
        with gzip.open(gz_fn, 'rb') as f_in, open(gz_fn[:-3], 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

# 1. Load File

In [ ]:
present_transcripts_df=pd.read_csv('../Data/expression_by_probe.csv', index_col=0)

In [ ]:
probe_key=pickle.load(open('../Data/probe_key.p', 'rb'))

In [ ]:
sample_key_abb={
    'GSM215347':'Control 1',
    'GSM215348':'Control 2',
    'GSM215349':'Control 3',
    'GSM215350':'Activated with IL2 rep 1',
    'GSM215351':'Activated with IL2 rep 2',
    'GSM215352':'Activated with IL2 rep 3',
    'GSM215353':'Activated with IL15 rep 1',
    'GSM215354':'Activated with IL15 rep 2',
    'GSM215355':'Activated with IL15 rep 3',
    'GSM215356':'Activated with IL21 rep 1',
    'GSM215357':'Activated with IL21 rep 2',
    'GSM215358':'Activated with IL21 rep 3'
}

experimental_groups={
    'Control': ['GSM215347', 'GSM215348', 'GSM215349'],
    'IL2': ['GSM215350', 'GSM215351', 'GSM215352'],
    'IL15': ['GSM215353', 'GSM215354', 'GSM215355'],
    'IL21': ['GSM215356', 'GSM215357', 'GSM215358'],
}

# 3. PCA

Principal Component Analysis is a popular method of visualizing variance in gene expression data and has many uses in statistics and machine learning. If you want to know more about PCA, this [excerpt from the Python Data Science Handbook by Jake VanderPlas](https://jakevdp.github.io/PythonDataScienceHandbook/05.09-principal-component-analysis.html) has a nice explanation.

In [ ]:
pca=decomposition.PCA(n_components=3)
components=pca.fit_transform(present_transcripts_df.T.values)
components_df=pd.DataFrame(components, index=present_transcripts_df.T.index)

We want a high combined explained variance ratio. If there we can't get a high explained variance ratio within the first 2-3 components, we should choose a different method

In [ ]:
pca.explained_variance_ratio_

In [ ]:
pca.explained_variance_ratio_.sum()

In [ ]:
components_df

### 3D Scatterplot - visualize PCA

In [ ]:
from sciomics_functions import grouped_scatterplot_3d

With 3 components, we can visualize them in three dimensions or two with a simple scatterplot showing different components as the axes.

In [ ]:
grouped_scatterplot_3d(components_df, experimental_groups, 'rgyb', '../results/3d_pca.png')

In [ ]:
# Make a bunch of 2D scatterplots instead of one 3D scatterplot

for combo in [(0,1), (0,2), (1,2)]:
    
    fig=plt.figure(figsize=(5,5), dpi=100)
    ax=fig.gca()
    
    #Go through each experimental group and plot in a different color
    for i, j in zip(experimental_groups, 'rygb'):
        
        xs=components_df.loc[experimental_groups[i], combo[0]].values
        ys=components_df.loc[experimental_groups[i], combo[1]].values
    
        ax.scatter(xs, ys, c=j, s=400)
        fig.savefig('../results/2d_pca_{}.png'.format(combo))

# 4. One-way ANOVA/F-test on 4 conditions

We need to change the format of our data into a format that statistical analysis functions can accept. Different functions take data in different forms. To find out what format you need to change your data to, read the documentation for the function. In this case, we are using statsmodel's [one way ANOVA F-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html).

In [ ]:
#Separate expression dataframe by experimental groups 
experimental_groups_expression={}
for condition in experimental_groups:
    experimental_groups_expression[condition]=present_transcripts_df[experimental_groups[condition]].T

In [ ]:
F, p=f_oneway(experimental_groups_expression['Control'], 
               experimental_groups_expression['IL2'],
               experimental_groups_expression['IL15'],
               experimental_groups_expression['IL21'], axis=0)

In [ ]:
len(F)

In [ ]:
transcripts=list(present_transcripts_df.index)

F_p_df=pd.DataFrame(index=transcripts, data=zip(F, p), columns=['F', 'p'])

In [ ]:
#Dataframe with f statistic and p-value
F_p_df

# 5. Benjamini-Hochberg step up correction

This step is necessary to correct for multiple comparisons

In [ ]:
bh=multipletests(p, alpha=0.01, method='fdr_bh')
F_p_df['reject null']=bh[0]
F_p_df['corrected p']=bh[1]

In [ ]:
diff_exp_genes_df=F_p_df.loc[F_p_df['reject null']]

In [ ]:
diff_exp_genes_list=list(diff_exp_genes_df.index)

In [ ]:
len(diff_exp_genes_list)

# 5. Diff-exp A: Filter by fold change (No post-hoc)

## Calculating fold change vs control
Fold change is simply the value of the second condition divided by the value of the first condition.

In [ ]:
#Calculate mean by experimental group
condition_means={}
condition_means_diffexp={}

for i in experimental_groups:
    condition_means_diffexp[i]=present_transcripts_df.loc[diff_exp_genes_list,experimental_groups[i]].T.mean()
    condition_means[i]=present_transcripts_df.loc[:, experimental_groups[i]].T.mean()    
    
condition_means_df=pd.DataFrame(condition_means)
condition_means_diffexp_df=pd.DataFrame(condition_means_diffexp)

In [ ]:
#Calculate fold change
fold_change_dict={}
for i in condition_means_df:
    fold_change_dict[i]=(condition_means_df[i]/condition_means_df['Control'])
fold_change_df=pd.DataFrame(fold_change_dict)

In [ ]:
fold_change_df

## Filter for fold change magnitude

In [ ]:
diffexp_fold_change={}

for i in fold_change_df:
    
    if not i=='Control':
        diffexp_fold_change[i]={'up':[],
                      'dn':[]}
        
        #Filter for significance
        for j in diff_exp_genes_list:
            
            fold_change=fold_change_df.loc[j, i]
            
            if fold_change > 2:
                diffexp_fold_change[i]['up'].append(j)
                
            elif fold_change < 0.5:
                diffexp_fold_change[i]['dn'].append(j)

In [ ]:
#Count of how many differentially expressed probes up and downregulated in each group

print("Differentially expressed probes in each group:")
for i in diffexp_fold_change:
    for j in diffexp_fold_change[i]:
        print ('{}_{}: {}'.format(i, j, len(diffexp_fold_change[i][j])))

In [ ]:
#Map probe names to gene names

diffexp_genes_fold_change={}

for condition in diffexp_fold_change:
    
    diffexp_genes_fold_change[condition]={}
    
    for up_or_dn in diffexp_fold_change[condition]:
        
        #There are probes that map to multiple gene symbols, so we use nested list comprehension and change it to a 
        #set to remove duplicates
        
        diffexp_genes_fold_change[condition][up_or_dn]=set([gene for i in diffexp_fold_change[condition][up_or_dn] 
                                                        for gene in probe_key[i]])

In [ ]:
print("Differentially expressed genes in each group")

for i in diffexp_genes_fold_change:
    for j in diffexp_genes_fold_change[i]:
        print('{}_{}: {}'.format(i, j, len(diffexp_genes_fold_change[i][j])))

# Calculate Pearson Correlation

In [ ]:
from sciomics_functions import matrix_pearson

In [ ]:
pearson, p=matrix_pearson(present_transcripts_df)

In [ ]:
pearson.columns=[sample_key_abb[i] for i in pearson.columns]

pearson.index=pearson.columns

In [ ]:
sns.heatmap(pearson)
plt.savefig('../results/Pearson_clustermap.png')

# Compare with Gene Sets
This study produced 20 gene sets on the Molecular Signatures Database (MSigDB). We can get the genes from those gene sets and compare them to the genes we found to be differentially expressed

In [ ]:
#Keeping track of what condition each gene set measures, since the names are confusing

gene_sets={
    'GSE8685_IL15_ACT_IL2_STARVED_VS_IL21_ACT_IL2_STARVED_CD4_TCELL_UP': 'IL15_IL21_up',
    'GSE8685_IL15_ACT_IL2_STARVED_VS_IL21_ACT_IL2_STARVED_CD4_TCELL_DN': 'IL15_IL21_dn',
    'GSE8685_IL2_ACT_IL2_STARVED_VS_IL15_ACT_IL2_STARVED_CD4_TCELL_DN': 'IL15_Control_dn',
    'GSE8685_IL2_ACT_IL2_STARVED_VS_IL15_ACT_IL2_STARVED_CD4_TCELL_UP': 'IL15_Control_up',
    'GSE8685_IL2_ACT_IL2_STARVED_VS_IL21_ACT_IL2_STARVED_CD4_TCELL_DN': 'IL2_IL21_dn',
    'GSE8685_IL2_ACT_IL2_STARVED_VS_IL21_ACT_IL2_STARVED_CD4_TCELL_UP': 'IL2_IL21_up',
    'GSE8685_IL2_STARVED_VS_IL15_ACT_IL2_STARVED_CD4_TCELL_DN': 'IL15_Control_dn',
    'GSE8685_IL2_STARVED_VS_IL15_ACT_IL2_STARVED_CD4_TCELL_UP': 'IL15_Control_up',
    'GSE8685_IL2_STARVED_VS_IL21_ACT_IL2_STARVED_CD4_TCELL_DN': 'IL21_Control_dn',
    'GSE8685_IL2_STARVED_VS_IL21_ACT_IL2_STARVED_CD4_TCELL_UP': 'IL21_Control_up',
    'GSE8685_IL2_STARVED_VS_IL2_ACT_IL2_STARVED_CD4_TCELL_DN': 'IL2_Control_dn',
    'GSE8685_IL2_STARVED_VS_IL2_ACT_IL2_STARVED_CD4_TCELL_UP': 'IL2_Control_up',
    'IL15_UP.V1_DN':'IL15_Control_dn',
    'IL15_UP.V1_UP':'IL15_Control_up',
    'IL21_UP.V1_DN':'IL21_Control_dn',
    'IL21_UP.V1_UP':'IL21_Control_up',
    'IL2_UP.V1_DN':'IL2_Control_dn',
    'IL2_UP.V1_UP':'IL2_Control_up',
    'MARZEC_IL2_SIGNALING_DN': 'IL2_Control_dn',
    'MARZEC_IL2_SIGNALING_UP':'IL2_Control_up',
}

In [ ]:
# Load python dictionary containing each gene set's genes
gene_sets_dict=pickle.load(open('../data/msigdb_marzec_gene_sets.p', 'rb'))

In [ ]:
from matplotlib_venn import venn2

In [ ]:
for gene_set in gene_sets:
    condition=gene_sets[gene_set].split('_')
    up_or_dn=condition[2]
    if condition[1] == 'Control':
        
        
        gene_set_genes=set(gene_sets_dict[gene_set])
        diff_exp_genes=set(diffexp_genes_fold_change[condition[0]][up_or_dn])
        
        #Plot venn diagram
        fig=plt.figure()
        ax=plt.gca()
        venn2([gene_set_genes, diff_exp_genes], set_labels=[gene_set, 'our genes'], ax=ax)

# Diff-exp B: Post-Hoc Analysis by using Pairwise Tukey
An F-test will tell us whether there is a significant difference across several conditions, but it doesn't tell us which conditions are different. Typically we want to do a post hoc test to tell us which conditions are significantly different from one another.

In [ ]:
from sciomics_functions import tukeyhsd_get_significant_comparisons

In [ ]:

post_hoc_diff_dict={}
post_hoc_dict={}

for transcript in diff_exp_genes_list:
    
    #Statsmodels' pairwise_tukeyhsd returns very difficult to interpret results, so we have another function to extrac the 
    #relevant information from these results
    tukey_summary=pairwise_tukeyhsd(present_transcripts_df.loc[transcript].to_numpy(), ['Control']*3+['IL2']*3+['IL15']*3+['IL21']*3, alpha=0.01)
    
    tukey_results=tukeyhsd_get_significant_comparisons(tukey_summary, include_CI=True)
    
    #We also want the non significant comparisons so we can make a volcano plot later
    tukey_results_w_rejected=tukeyhsd_get_significant_comparisons(tukey_summary, include_CI=True, only_sig_comparisons=False)
    
    #Make df with only significant results so we can look at them
    for comparison in tukey_results.index:
        if not comparison in post_hoc_diff_dict:
            post_hoc_diff_dict[comparison]={}
        post_hoc_diff_dict[comparison][transcript]=tukey_results.loc[comparison, 'p-adj']        
    post_hoc_diff_df=pd.DataFrame(post_hoc_diff_dict)
    
    #Make a df with rejected transcript included for later volcano plots    
    for comparison in tukey_results_w_rejected.index:
        if not comparison in post_hoc_dict:
            post_hoc_dict[comparison]={}
        post_hoc_dict[comparison][transcript]=tukey_results_w_rejected.loc[comparison, 'p-adj']
    post_hoc_df=pd.DataFrame(post_hoc_dict)

In [ ]:
print("Significant differentially expressed genes after post hoc analysis:")

for condition in post_hoc_diff_dict:
    print('{}: {} genes'.format(condition, len(post_hoc_diff_dict[condition])))

### Filter for Fold Change 2 or higher

In [ ]:
post_hoc_fold_change_dict={}

#Do for each comparison
for comparison in post_hoc_diff_dict:
    
    condA=comparison.split('_')[0]
    condB=comparison.split('_')[1]
    
    #Calculate fold change
    post_hoc_fold_change_dict[comparison]=(present_transcripts_df.loc[:,experimental_groups[condB]].T.mean()/present_transcripts_df.loc[:,experimental_groups[condA]].T.mean())
post_hoc_fold_change_df=pd.DataFrame(post_hoc_fold_change_dict)

In [ ]:
diffexp_fold_change_ph={}
  
for condition in post_hoc_diff_dict:

    diffexp_fold_change_ph[condition]={
        'up':[],
        'dn':[]
    }
    for transcript in post_hoc_diff_dict[condition]:
        
        #Note that this fold change is for condition 2 relative to condition 1 in 1_2
        fold_change=post_hoc_fold_change_df.loc[transcript, condition]
        if fold_change > 2:
            diffexp_fold_change_ph[condition]['up'].append(transcript)

        elif fold_change < 0.5:
            diffexp_fold_change_ph[condition]['dn'].append(transcript)


In [ ]:
#Change probe names to gene names

diffexp_genes_fold_change_ph={}

for condition in diffexp_fold_change_ph:
    
    diffexp_genes_fold_change_ph[condition]={}
    
    for up_or_dn in diffexp_fold_change_ph[condition]:
        
        #There are probes that map to multiple gene symbols, so we use nested list comprehension and change it to a 
        #set to remove duplicates
        diffexp_genes_fold_change_ph[condition][up_or_dn]=set([gene for i in diffexp_fold_change_ph[condition][up_or_dn] 
                                                        for gene in probe_key[i]])

In [ ]:
for condition in diffexp_genes_fold_change_ph:
    for up_or_dn in diffexp_genes_fold_change_ph[condition]:
        print('{}_{}: {} genes'.format(condition, up_or_dn, len(diffexp_genes_fold_change_ph[condition][up_or_dn])))

## Compare with gene sets

In [ ]:
gene_sets

In [ ]:
for gene_set in gene_sets:
    condition=gene_sets[gene_set].split('_')
    up_or_dn=condition[2]
    
    #Only comparing fold change from control
    if condition[1] == 'Control':
        
        gene_set_genes=set(gene_sets_dict[gene_set])
        diff_exp_genes=set(diffexp_genes_fold_change_ph['{}_{}'.format(condition[1], condition[0])][up_or_dn])

        #Plot venn diagrams
        fig=plt.figure()
        ax=plt.gca()
        venn2([gene_set_genes, diff_exp_genes], set_labels=[gene_set, 'our genes'], ax=ax)

# Compare ANOVA with no post-hoc analysis to ANVOA with post-hoc analysis (Remove ?)

In [ ]:
for comparison in diffexp_fold_change_ph:
    condition=comparison.split('_')
    if condition[0]=='Control':
        for up_or_dn in ['up', 'dn']:
        
            deg_a=set(diffexp_fold_change[condition[1]][up_or_dn])
            deg_b=set(diffexp_fold_change_ph[comparison][up_or_dn])
            
            print('{}_{}'.format(comparison, up_or_dn))
            
            fig=plt.figure()
            ax=plt.gca()
            venn2([deg_a, deg_b], set_labels=['No post-hoc', 'With post-hoc'], ax=ax)            

# Create Volcano Plots (Using post-hoc p-values)

In [ ]:
#For each condition, make a dataframe with the log(fold change) and -log(p) from the post hoc p-value
for condition in post_hoc_fold_change_df:
    
    volcano_plot_df=pd.DataFrame([pd.Series(np.log2(post_hoc_fold_change_df[condition]), name='log2(Fold Change)'), 
                                  pd.Series(-np.log10(post_hoc_df[condition]), name='-log(p)')]
                                 )
    
    #volcano_plot_df.columns=[probe_key[i] for i in volcano_plot_df.columns]
    
    volcano_plot_df=volcano_plot_df.T
    
    volcano_plot_df['hover_text_labels']=volcano_plot_df.index
    
    fig=px.scatter(volcano_plot_df, x='log2(Fold Change)', y='-log(p)', hover_data=['hover_text_labels'], title=condition)
    fig.show()

# Probe IL2/15/21 Essentiality in DepMap/CCLE

In [ ]:
dm_sample_info=pd.read_csv('../data/dm_sample_info.csv', index_col=0)
dm_gene_expression=pd.read_csv('../data/CCLE_expression.csv', index_col=0)

In [ ]:
#cks stands for cytokines
cks=['IL2', 'IL15', 'IL21']

In [ ]:
lineage=dm_sample_info['lineage']

In [ ]:
dm_gene_expression.columns=[i.split(' ')[0] for i in dm_gene_expression.columns]

In [ ]:
dm_gene_expression_oi=dm_gene_expression.loc[:,cks]

In [ ]:
dependency=pd.read_csv('../data/CRISPR_gene_dependency.csv', index_col=0)

In [ ]:
dependency

### Make one histogram with all cytokines

In [ ]:
#append values into a format plotly express will take

values=[]
labels=[]
index=[]

for ck in cks:
    values+=[*dm_gene_expression[ck].values]
    labels+=[ck]*dm_gene_expression[ck].shape[0]

In [ ]:
big_px_df=pd.DataFrame([values, labels], index=['gene_expression', 'gene'], dtype=float)

In [ ]:
px.histogram(data_frame=big_px_df.T, x='gene_expression', color='gene', barmode='overlay', title="Average Cytokine Gene Expression")

In [ ]:
dm_gene_expression

In [ ]:
#Group by mean gene expression p
dm_ge_x_lineage=dm_gene_expression.groupby(lineage).mean()

In [ ]:
values=[]
labels=[]
index=[]

for ck in cks:
    values+=[*dm_ge_x_lineage[ck].values]
    labels+=[ck]*dm_ge_x_lineage[ck].shape[0]
    index+=[*dm_ge_x_lineage.index]

In [ ]:
big_px_df=pd.DataFrame([values, labels, index], index=['average_gene_expression', 'gene', 'tissue_lineage'])

In [ ]:
px.bar(data_frame=big_px_df.T, x='tissue_lineage', y='average_gene_expression', color='gene', barmode='overlay', title='Average Cytokine Gene Expression by Tissue Lineage', height=1000)

In [ ]:
#append values into a format plotly express will take

values=[]
labels=[]
index=[]

for ck in cks:
    values+=[*dependency[ck].values]
    labels+=[ck]*dependency[ck].shape[0]

In [ ]:
big_px_df=pd.DataFrame([values, labels], index=['gene_dependency_score', 'gene'])

In [ ]:
px.histogram(data_frame=big_px_df.T, x='gene_dependency_score', color='gene', barmode='overlay', title='Cytokine Gene Dependency Score')

In [ ]:
dm_dep_x_lineage=dependency.groupby(lineage).mean()

In [ ]:
values=[]
labels=[]
index=[]

for ck in cks:
    values+=[*dm_dep_x_lineage[ck].values]
    labels+=[ck]*dm_dep_x_lineage[ck].shape[0]
    index+=[*dm_dep_x_lineage.index]

In [ ]:
big_px_df=pd.DataFrame([values, labels, index], index=['average_gene_dependency', 'gene', 'tissue_lineage'])

In [ ]:
px.bar(data_frame=big_px_df.T, x='tissue_lineage', y='average_gene_dependency', color='gene', barmode='overlay', title='Average Cytokine Gene  Dependency Score by Tissue Lineage', height=1000)

### or make 3 different histograms - to see different things

In [ ]:
dm_gene_expression=dm_gene_expression.T

In [ ]:
for ck in cks:

    fig=px.histogram(data_frame=dm_gene_expression.loc[ck].to_frame(), x=ck, range_x=[0,8], nbins=200, title='{}: {}'.format(ck, 'Gene Expression'), labels={ck: 'Gene Expression Value'})

    fig.show()
    
    fig=px.bar(dm_gene_expression.T.groupby(lineage).mean().loc[:, ck], title='Average {} Expression by Tissue Lineage'.format(ck))
    fig.show()

In [ ]:
dependency=dependency.T

In [ ]:
for ck in cks:

    fig=px.histogram(x=dependency.loc[ck].values, range_x=[0,1], nbins=200, title='{}: {}'.format(ck, 'Dependency'))

    fig.show()
    
    fig=px.bar(dependency.T.groupby(lineage).mean().loc[:, ck], title='Average {} Dependency by Tissue Lineage'.format(ck))
    fig.show()